가중치를 랜덤으로 초기화 하는 이유 : 어떤 값을 고정해서 주면 값이 엄청 커지거나 소멸될 수 있다.

-----

### \#\# 1. [가중치 초기화] Xavier 초기화 기법 적용


신경망의 성능을 높이기 위해, 이전 계층과 다음 계층의 노드 수를 고려하는 **'Xavier 초기화'** 기법을 적용해 보세요. 입력 노드가 784개, 출력 노드가 256개일 때, 이 기법에 따라 가중치 행렬을 정규분포를 이용해 생성해 보세요.

**파이썬 코드:**

입력 갯수에 따라 가중치의 값을 정하자<br>
갯수와 가중치의 관계는 반비례 따라서 생각해낸것이 1/in

In [ ]:
import numpy as np

# 입력과 출력 노드의 수 정의
n_in = 784  # 예: MNIST 이미지 크기 (28*28)
n_out = 256 # 예: 은닉층의 노드 수

# Xavier 초기화에 따른 표준편차 계산
std_dev = np.sqrt(2/n_in + n_out)   # 여기

# 계산된 표준편차를 사용하여 가중치 행렬 생성 (정규분포)
weights = np.random.randn(n_in, n_out) * std_dev

print("Xavier 초기화 기법 적용")
print(f"입력 노드 수: {n_in}, 출력 노드 수: {n_out}")
print(f"생성된 가중치 행렬의 shape: {weights.shape}")
print(f"mean: {weights.mean():.4f}, std: {weights.std():.4f}")


**관련 인공지능 개념:**
**Xavier(=Glorot) 초기화:** 일반적인 무작위 초기화보다 더 발전된 기법입니다. 신경망의 계층이 깊어지면서 신호가 너무 강해지거나 약해지는 것을 막기 위해, **입력과 출력 노드의 수를 고려하여** 초기 가중치의 분포를 조절합니다. 이는 학습을 더 안정적이고 빠르게 만듭니다. `np.random.randn`은 표준 정규분포(평균 0, 표준편차 1)를 따르는 난수를 생성합니다.



-----

### \#\# 2. [데이터 증강] 여러 변형을 동시에 적용하기

32x32 크기의 컬러 이미지(3채널) 10장으로 구성된 데이터 배치(10, 32, 32, 3)가 있다고 가정합니다. 이 이미지들에 대해 50% 확률로 **좌우 반전**을 적용하고, 동시에 **약한 가우시안 노이즈(평균0, 표준편차 0.05)**를 추가하는 데이터 증강 코드를 작성해 보세요.

**파이썬 코드:**

In [2]:
import numpy as np

# (장수, 높이, 너비, 채널) 형태의 가상 이미지 데이터 배치
image_batch = np.random.rand(10, 32, 32, 3)


# 1. 무작위 좌우 반전 적용
for i in range(image_batch.shape[0]):
    # 50% 확률로 좌우 반전 실행
    if np.random.rand() > 0.5:         #여기
        # np.fliplr()은 2차원 배열에만 적용되므로, 각 채널별로 적용
        image_batch[i] = np.array([np.fliplr(channel) for channel in image_batch[i].T]).T  #여기

# 2. 가우시안 노이즈 추가(평균이 0이고 표준편차가 0.05인 정규분포(가우시안) 노이즈)
noise = np.random.normal(0, 0.05, image_batch.shape)
augmented_batch = image_batch + noise
#np.clip(배열, 최소값, 최대값)**은 배열의 모든 값이 지정된 범위 안에 들어가도록 강제로 조정
augmented_batch = np.clip(augmented_batch, 0, 1)

print("데이터 증강 적용 후 배치의 shape:", augmented_batch.shape)

데이터 증강 적용 후 배치의 shape: (10, 32, 32, 3)


/tmp/ipython-input-1628477481.py:10: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.random.rand(0, 1) > 0.5:         #여기


**관련 인공지능 개념:**
**데이터 증강 파이프라인:** 실제 딥러닝에서는 **여러 증강 기법을 확률적으로 조합하여** 파이프라인을 만듭니다. `np.random.rand()`로 확률 조건을 만들어 특정 변환을 적용할지 말지를 결정함으로써, 매 학습마다 모델이 조금씩 다른 데이터를 보도록 하여 일반화 성능을 극대화합니다.



In [ ]:
#np.fliplr() 연습
#배열의 열(column)의 순서를 거꾸로 만듭니다. 첫 번째 열은 마지막 열이 되고,
#마지막 열은 첫 번째 열이 됩니다. 행(row)의 순서는 그대로 유지
import numpy as np
A = np.arange(15).reshape(3,5)
print(A)
print(np.fliplr(A))

-----

### \#\# 3. [드롭아웃] 실제 연산에 적용하기

10개의 노드를 가진 은닉층의 출력(`layer_output`)이 있다고 가정합니다. 이 출력에 50% 비율의 드롭아웃을 적용한 후, 다음 층으로 전달하기 전 \*\*남아있는 노드들의 신호를 보존(Inverted Dropout)\*\*하기 위해 값을 조정해 보세요.

**파이썬 코드:**

드롭아웃을 하는 이유는 과적합을 없애기 위해 에이스 친구들을 죽여서 버스타는 친구들 훈련시키기<br>
그리고 그 모듈들을 죽이고 나면 힘이 죽기 때문에 보존하기 위해 3번을 실시

In [ ]:
import numpy as np

p_dropout = 0.5
# 10개 노드에서 나온 가상의 출력값
layer_output = np.random.rand(10)
print(f"원본 출력: {layer_output.round(2)}")
# 1. 드롭아웃 마스크 생성
dropout_mask = (np.random.rand(10) > p_dropout)

# 2. 드롭아웃 적용 (비활성화된 노드는 0으로)
dropout_output = 1 / (1 - p_dropout) #여기
print(f"조정 전 출력: {dropout_output.round(2)}")

# 3. 신호 보존 (Inverted Dropout): 살아남은 노드들의 값을 p_dropout으로 나눠 크기를 키워줌
# 훈련 시에만 적용하고, 테스트 시에는 적용하지 않음
dropout_output =                     #여기
print(f"드롭아웃 마스크: {dropout_mask.astype(int)}")
print(f"조정된 최종 출력: {dropout_output.round(2)}")

**관련 인공지능 개념:**
**Inverted Dropout:** 드롭아웃을 적용하면 출력 신호의 전체 크기가 줄어듭니다. 이를 보상하기 위해, 훈련 시 살아남은 노드들의 값을 **`1/(1-p_dropout)`만큼 곱해줍니다.** 이렇게 하면 훈련 때와 테스트 때의 출력 크기 기댓값이 같아져, 테스트 시에는 드롭아웃을 적용하지 않아도 일관된 결과를 얻을 수 있습니다.



-----

### \#\# 4. [미니배치] 배치를 생성하는 제너레이터 만들기


전체 데이터셋을 한 번 학습하는 것을 1 에폭(epoch)이라고 합니다. 1000개의 데이터를 32개씩 미니배치로 나눌 때, 1 에폭 동안 **데이터를 무작위로 섞고 순서대로 미니배치를 생성**하는 파이썬 제너레이터(generator) 함수를 만들어 보세요.

**파이썬 코드:**

토기의 최저 위치를 찾을때 개미를 몽땅 넣는 방법, 하나하나 씩 릴레이 하는 방법 이 두가지 사이 방법이다.

In [ ]:
import numpy as np

def batch_generator(data, batch_size=32, shuffle=True):
    """미니배치를 생성하는 제너레이터 함수"""
    indices = np.arange(len(data))
    if shuffle:
        np.random.shuffle(indices)   #여기

    for start_idx in range(0, len(data), batch_size):
        end_idx = min(start_idx + batch_size, len(data))
        batch_indices = indices[start_idx:end_idx]
        yield data[batch_indices]

# 0부터 999까지의 가상 데이터
full_data = np.arange(1000)

gen = batch_generator(full_data, batch_size=32)

# 첫 번째 배치 출력
print("\n--- 첫 번째 배치 ---")
first_batch = next(gen)   #여기
print(f"첫 번째 배치의 내용 (일부): {first_batch[:5]}")

# 두 번째 배치 출력 (새로 생성하지 않고 이어서 호출)
print("\n--- 두 번째 배치 ---")
second_batch = next(gen)  #여기
print(f"두 번째 배치의 내용 (일부): {second_batch[:5]}")

**관련 인공지능 개념:**
**데이터 로더/제너레이터:** 대용량 데이터셋은 메모리에 한 번에 올릴 수 없습니다. 따라서 학습 시 필요한 만큼의 **미니배치를 동적으로 생성**하여 메모리를 효율적으로 사용합니다. `np.random.shuffle()`로 매 에폭마다 데이터 순서를 섞어주면, 모델이 데이터 순서에 과적합되는 것을 방지하고 학습 안정성을 높일 수 있습니다.



In [ ]:
# 위의 코드를 2번째 배치까지 출력하되 다음과 같이 출력되도록 해보자.내용은 다를 수 있음.
#--- 첫 번째 배치 ---
#start_idx: 0
#첫 번째 배치의 내용 (일부): [170 813 654 812 621]

#--- 두 번째 배치 ---
#start_idx: 32
#두 번째 배치의 내용 (일부): [962 149 978 425 544]

# 원하는 개수 만큼 미니 배치 생성(퀴즈)

In [3]:
#원하는 미니배치 개수 만큼(여기서는 5)
import numpy as np
import itertools

def batch_generator(data, batch_size=32, shuffle=True):
  """미니배치를 생성하는 제너레이터 함수"""
  indices = np.arange(len(data))
  if shuffle:
      np.random.shuffle(indices)

  for start_idx in range(0, len(data), batch_size):
      end_idx = min(start_idx + batch_size, len(data))
      batch_indices = indices[start_idx:end_idx]
      yield data[batch_indices]

# 0부터 999까지의 가상 데이터
full_data = np.arange(1000)

# 생성할 미니배치의 개수 설정
num_batches_to_generate = 5

# 제너레이터 생성
gen = batch_generator(full_data, batch_size=32)

# 원하는 개수만큼 미니배치 생성
generated_batches = list(itertools.islice(gen, num_batches_to_generate)) #괄호안 채우기, itertools.islice() 활용

print(f"{num_batches_to_generate}개의 미니배치 생성 완료:")
for i, batch in enumerate(generated_batches):
  print(f"--- 배치 {i+1} ---")
  print(f"크기: {batch.shape}")
  print(f"내용 (일부): {batch[:5]}")

5개의 미니배치 생성 완료:
--- 배치 1 ---
크기: (32,)
내용 (일부): [285 478 555  42 245]
--- 배치 2 ---
크기: (32,)
내용 (일부): [887 979 856  40 228]
--- 배치 3 ---
크기: (32,)
내용 (일부): [100 947 966  26  85]
--- 배치 4 ---
크기: (32,)
내용 (일부): [701 523 263 310 417]
--- 배치 5 ---
크기: (32,)
내용 (일부): [603 914 859 575 928]


-----

### \#\# 5. [강화학습] 간단한 Q-러닝 업데이트


4개의 상태와 2개의 행동이 있는 환경에서, 에이전트가 \*\*무작위 행동(탐험)\*\*을 통해 얻은 보상을 바탕으로 자신의 \*\*행동 가치 테이블(Q-table)\*\*을 업데이트하는 과정을 시뮬레이션해 보세요.

**파이썬 코드:**

In [ ]:
import numpy as np

# Q-table 초기화 (4개 상태, 2개 행동)
q_table = np.zeros((4, 2))
print(f"\n업데이트 전 Q-table:\n", q_table.round(2))

learning_rate = 0.1 # 학습률
discount_factor = 0.9 # 미래 보상 할인율

# 현재 상태
current_state = np.random.randint(0, 4)
print(f"current_state:{current_state}") #

# 탐험: 현재 상태에서 무작위 행동 선택
action = np.random.randint(0, 2)
print(f"action:{action}") #

# 무작위 행동의 결과로 다음 상태와 보상이 주어졌다고 가정
next_state = np.random.randint(0, 4)
reward = np.random.rand()
print(f"next_state:{next_state}") #
print(f"reward:{reward}") #

# Q-러닝 업데이트 공식
old_value = q_table[current_state, action]
next_max = np.max(q_table[next_state])
new_value = old_value + learning_rate * (reward + discount_factor * next_max - old_value)  #여기
print(f"old_value:{old_value}")#
print(f"next_max:{next_max}")#
print(f"new_value:{new_value}")#


print(f"\n상태 {current_state}에서 행동 {action}을 수행!")
print(f"결과: 다음 상태={next_state}, 보상={reward:.2f}")
print(f"\n업데이트 후 Q-table:\n", q_table.round(2))

**관련 인공지능 개념:**
**Q-러닝 (Q-Learning):** 강화학습의 대표적인 알고리즘으로, 에이전트는 **Q-table**이라는 표에 '각 상태에서 각 행동을 했을 때 얼마나 좋은가'를 기록합니다. `np.random`으로 무작위 행동(탐험)을 하며 얻은 \*\*경험(보상)\*\*을 바탕으로 이 Q-table을 점진적으로 업데이트하여, 결국 최적의 행동 정책을 학습하게 됩니다.





-----

### \#\# 6. [데이터 샘플링] 오버샘플링과 언더샘플링 조합하기

1000개의 데이터 중 950개가 다수 클래스(0), 50개가 소수 클래스(1)인 불균형 데이터가 있습니다. 다수 클래스는 100개로 **랜덤 언더샘플링**하고, 소수 클래스는 기존 데이터에 약간의 **노이즈를 추가하여 100개로 랜덤 오버샘플링**하여 균형 잡힌 데이터셋을 만들어 보세요.

**파이썬 코드:**

In [ ]:
import numpy as np

# 가상 불균형 데이터 생성 (feature 5개)
X = np.random.rand(1000, 5)
y = np.array([0] * 950 + [1] * 50)

# 1. 클래스별 인덱스 분리
major_indices = np.where(y == 0)[0]
minor_indices = np.where(y == 1)[0]

# 2. 다수 클래스 언더샘플링
undersampled_major_indices = np.random.choice(major_indices, 100, replace=False)

# 3. 소수 클래스 오버샘플링 (부족해서 추가를 원하는 개수 만큼 만들어 노이즈 추가)
num_to_add = 100 - len(minor_indices)
oversample_indices = np.random.choice(minor_indices, num_to_add, replace=False)     #여기
X_oversampled = X[oversample_indices] + np.random.normal(  ,     , (1, 5))  #feature 5개

# 4. 데이터 합치기
X_balanced = np.vstack(            ,            ,            )  #여기
y_balanced = np.array([0] * 100 + [1] * 100)


print(f"원본 데이터 shape: {X.shape}, 레이블 비율: {np.bincount(y)}")
print(f"균형 맞춘 데이터 shape: {X_balanced.shape}, 레이블 비율: {np.bincount(y_balanced)}")

**관련 인공지능 개념:**
**하이브리드 샘플링 (Hybrid Sampling):** 언더샘플링은 정보 손실의 위험이 있고, 오버샘플링은 과적합의 위험이 있습니다. **두 기법을 조합**하면 이러한 단점을 완화할 수 있습니다. 특히 소수 클래스를 단순히 복제하는 대신, 원본에 **무작위 노이즈**를 추가하여 새로운 데이터를 생성하는 방식은 **SMOTE**와 같은 고급 오버샘플링 기법의 기본 원리가 됩니다.



-----

### \#\# 7. [하이퍼파라미터 탐색] 탐색 결과 저장 및 최적 값 찾기

학습률과 은닉층 크기를 무작위로 10번 탐색하며, 각 조합에 대한 가상의 검증 정확도(validation accuracy)를 생성합니다. 모든 탐색 결과를 기록한 후, **가장 높은 정확도를 보인 하이퍼파라미터 조합**을 찾아 출력해 보세요. (코드에서는 실제 훈련할 수 없으니 랜덤으로 가상의 정확도를 80%~99% 되도록 생성)   

learning_rate는 $10^{-4}$ ~ $10^{-1}$  

hidden_size는 [32,64,128,256]중에 랜덤 선택하도록

**파이썬 코드:**

In [ ]:
import numpy as np

num_searches = 10
results = []

for i in range(num_searches):
    learning_rate =                # 여기,  np.random.uniform 사용해서 0.0001 ~ 0.1
    hidden_size =                   # 여기,  후보군 중에서 선택

    # 실제로는 모델을 훈련시킨 후 얻는 값
    validation_accuracy =                        # 여기, np.random.rand()사용해서 80% ~ 99% 사이의 가상 정확도

    results.append({
        'learning_rate': learning_rate,
        'hidden_size': hidden_size,
        'accuracy': validation_accuracy
    })

# 정확도를 기준으로 내림차순 정렬
sorted_results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

best_params = sorted_results[0]

print(f"가장 높은 정확도: {best_params['accuracy']:.4f}")
print("최적 하이퍼파라미터:")
print(f"  - Learning Rate: {best_params['learning_rate']:.5f}")
print(f"  - Hidden Size: {best_params['hidden_size']}")

**관련 인공지능 개념:**
**베이즈 최적화 (Bayesian Optimization):** 무작위 탐색은 이전 탐색 결과와 상관없이 매번 무작위로 값을 선택합니다. 여기서 더 나아가, **이전 탐색 결과들을 바탕으로** 다음 탐색 지점을 더 지능적으로 선택하는 방법을 베이즈 최적화라고 합니다. 위 코드는 이러한 고급 기법의 기초가 되는 **탐색 결과를 기록하고 최적 값을 찾는 과정**을 보여줍니다.



-----

### \#\# 8. [몬테카를로] 고차원 공간의 부피 추정

2차원 원의 넓이를 구하는 것에서 확장하여, 한 변의 길이가 2인 정육면체 안에 내접한 **3차원 구(sphere)의 부피**를 몬테카를로 방법으로 추정해 보세요.
(실제 구의 부피는 $(\frac{4}{3})\pi r^3$ 입니다.)

**파이썬 코드:**

In [ ]:
import numpy as np

num_points = 100000

# -1과 1 사이의 3차원 좌표 (x, y, z)를 10만 개씩 생성
points = np.random.uniform(-1, 1, (num_points, 3))

# 원점으로부터의 거리 제곱 계산 (x^2 + y^2 + z^2)
distance_sq = np.sum(           , axis= ) #여기

# 거리가 1 이하인 점들 (구 안의 점)의 개수 세기
points_in_sphere = np.sum(             ) #여기

# 부피 추정
cube_volume = 2**3 # 8
sphere_volume_approx =                                    #여기, 랜덤수로 부피 추청

# 이론적인 부피 값
true_sphere_volume = (4/3) * np.pi * (1**3)

print(f"몬테카를로 추정 부피: {sphere_volume_approx:.4f}")
print(f"이론적인 실제 부피: {true_sphere_volume:.4f}")

**관련 인공지능 개념:**
**차원의 저주 (Curse of Dimensionality):** 데이터의 차원이 높아질수록 공간의 부피는 기하급수적으로 커져 데이터가 매우 희소(sparse)해지는 현상입니다. 몬테카를로 방법은 이러한 **고차원 공간의 부피나 적분을 근사 계산**하는 데 매우 효과적인 도구이며, 이는 복잡한 확률 분포를 다루는 최신 AI 모델에서 중요한 역할을 합니다.



-----

### \#\# 9. [확률 분포] 다변수 정규분포 생성

두 변수가 서로 **상관관계**를 가지는 2차원 데이터를 생성해야 할 때가 있습니다. 평균이 `[0, 0]`이고 특정 공분산 행렬을 따르는 **다변수 정규분포**에서 1000개의 샘플을 생성하고, 산점도(scatterplot)를 그려 데이터의 분포를 확인해 보세요.

**파이썬 코드:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 평균 벡터와 공분산 행렬 정의
# 공분산이 양수이므로, x가 증가할 때 y도 증가하는 경향을 보임
mean = [0, 0]
cov_matrix = [[1, 0.8],  # [[var(x), cov(x,y)],
              [0.8, 1]]  #  [cov(y,x), var(y)]]

# 다변수 정규분포에서 1000x2 크기의 샘플 생성
samples = np.random.multivariate_normal(        ,        , 1000) #여기

# 산점도로 시각화
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.5)
plt.title("Multivariate Normal Distribution")
plt.xlabel("X value")
plt.ylabel("Y value")
plt.grid(True)
plt.axis('equal')
plt.show()

**관련 인공지능 개념:**
**다변수 정규분포 (Multivariate Normal Distribution):** 여러 확률 변수와 그들 사이의 상관관계를 함께 모델링하는 확률 분포입니다. 이는 **변수 간의 관계를 고려하여** 현실 세계와 더 유사한 데이터를 생성하거나, 불확실성을 모델링하는 데 사용됩니다. 예를 들어, 사람의 키와 몸무게처럼 서로 연관된 특징을 생성할 때 유용합니다.



-----

### \#\# 10. [데이터 셔플링] 입력과 레이블을 함께 섞기

이미지 데이터(`X`)와 해당 이미지의 정답(`y`)이 각각 다른 배열에 저장되어 있습니다. 두 배열의 순서를 섞을 때, **이미지와 정답 사이의 짝이 깨지지 않도록** 함께 셔플링하는 코드를 작성해 보세요.

**파이썬 코드:**

In [ ]:
import numpy as np

# 10개의 가상 데이터 (이미지)와 레이블 생성
X_data = np.arange(10).reshape(10, 1) * 10
y_labels = np.arange(10)

print("--- 셔플링 전 ---")
print(f"이미지 데이터: {X_data.flatten()}")
print(f"레이블:      {y_labels}")

# 1. 0부터 데이터 개수만큼의 인덱스를 생성
indices = np.arange(len(X_data))

# 2. 인덱스를 무작위로 섞음
np.random.shuffle(indices)
print(f"\n섞인 인덱스: {indices}")

# 3. 섞인 인덱스를 사용하여 데이터와 레이블을 재배열
X_shuffled = X_data[indices]
y_shuffled = y_labels[indices]

print("\n--- 셔플링 후 ---")
print(f"이미지 데이터: {X_shuffled.flatten()}")
print(f"레이블:      {y_shuffled}")

**관련 인공지능 개념:**
**데이터 정렬 및 인덱싱:** 실제 데이터셋에서는 입력 데이터와 정답 레이블이 쌍으로 관리됩니다. 학습 전에 데이터를 섞을 때, **데이터 자체가 아닌 인덱스를 섞은 후**, 이 **섞인 인덱스를 이용해 두 배열을 동일한 순서로 재배열**하는 것이 일반적이고 안전한 방법입니다. 이는 데이터와 레이블 간의 중요한 연결을 유지하면서 훈련 데이터의 무작위성을 보장합니다.